# Lab 3 - Nim

In [1]:
import logging
import random

In [2]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k

    def nimming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("Yeuch")

In [4]:
def nim_sum(elem: list):
  x = 0
  for e in elem:
    x = e ^ x
  return x

## Implementation

### Task 3.1 - An agent using fixed rules based on nim-sum
Based on the explanation available here: https://en.wikipedia.org/wiki/Nim  

It wants to finish every move with a nim-sum of 0, called 'secure position' (then it will win if it does not make mistakes). 

In [ ]:
def rule1(board, k):
  """
  Winning move if there is only one row with more than 1 obj
  If there is only one obj left, the current player lost
  """
  tmp = [(i, r) for i, r in enumerate(board) if r > 0]
  if len(tmp) == 1:
    row, num_obj = tmp[0]
    if num_obj == 1:
      return row, num_obj
    elif k or num_obj > k:
      return row, k
    else:
      return row, num_obj-1 #left to 1
  
raise NameError('Rule1 not applied')

In [ ]:
def rule2(board, k):
  """
  Current player on a insucure position -> is winning  
  --> Has to generate a secure position (bad for the other player)  
  """ 
  # Compute the nim-sum of all the heap sizes
  x = 0
  for r in board:
    x = r ^ x
  assert x >= 0

  if x > 0:
    # Find a heap where the nim-sum of X and the heap-size is less than the heap-size.
    # Then play on that heap, reducing the heap to the nim-sum of its original size with X
    
    for row, row_size in enumerate(board):
      if row_size == 0:
        break
      nim_sum = row_size ^ x  # nim-sum
      num_obj = abs(row_size - nim_sum)  # DA SISTEMARE <---------
      if nim_sum < row_size and (not k or num_obj <= k): # winning move
        return row, num_obj

  raise NameError('Rule2 not applied')

In [ ]:
def rule3(board, k):
  """
    [ nim-sum of x and heap-size == 0 or k forced a bad move to the player ]  
    Current player on a secure position or on a bad position bc of k -> is losing  
    --> Can only generate an insicure position (good for the other player)
  """
  not_zero_rows = [(i, row_size) for i, row_size in enumerate(board) if board[i] > 0]
  row, num_obj = random.choice(not_zero_rows)
  if not k or num_obj <= k:
    return row, num_obj # Clear the entire row
  else:
    return row, 1 # Take just one obj from the row

In [5]:
def action_task1(board, k):
  """
    The agent uses fixed rules based on nim-sum (expert-system)

    Returns the index of the pile and the number of pieces removed
  """
  # Winning move if there is only one row with more than 1 obj
  # If there is only one obj left, the current player lost
  tmp = [(i, r) for i, r in enumerate(board) if r > 0]
  if len(tmp) == 1:
    row, num_obj = tmp[0]
    if num_obj == 1:
      return row, num_obj
    elif k or num_obj > k:
      return row, k
    else:
      return row, num_obj-1 #left to 1


  # Compute the nim-sum of all the heap sizes
  x = 0
  for r in board:
    x = r ^ x
  assert x >= 0

  if x > 0:
    # Current player on a insucure position -> is winning
    # --> Has to generate a secure position (bad for the other player)
    # --> Find a heap where the nim-sum of X and the heap-size is less than the heap-size.
    # --> Then play on that heap, reducing the heap to the nim-sum of its original size with X
    
    for row, row_size in enumerate(board):
      if row_size == 0:
        break
      nim_sum = row_size ^ x  # nim-sum
      num_obj = abs(row_size - nim_sum)  # DA SISTEMARE <---------
      if nim_sum < row_size and (not k or num_obj <= k): # winning move
        return row, num_obj
  
  # x == 0 or k force a bad move to the player
  # Current player on a secure position or on a bad position bc of k -> is losing
  # --> Can only generate an insicure position (good for the other player)
  not_zero_rows = [(i, row_size) for i, row_size in enumerate(board) if board[i] > 0]
  row, num_obj = random.choice(not_zero_rows)
  if not k or num_obj <= k:
    return row, num_obj # Clear the entire row
  else:
    return row, 1 # Take just one obj from the row

### Task 3.2 - An agent using evolved rules

## Play a game

Choose the action and the game parameters

In [6]:
action = action_task1
num_rows = 5
k = 3

In [7]:
nim = Nim(num_rows, k)
player = 1

logging.info(f'Board:\t{nim._rows}')
logging.info(f'\tk = {nim._k}')

# Logs of the game
log_board = []
log_player = []
log_move = []

log_board.append(nim._rows)
log_player.append(-1)
log_move.append((-1,-1))

while not sum(nim._rows) == 0:
  player = int(not player) 
  row, num_obj = action(nim._rows, nim._k)
  nim.nimming(row, num_obj)

  log_board.append([x for x in nim._rows])
  log_player.append(player)
  log_move.append((row, num_obj))

logging.info(f'### Player {player} lost ###')

Board:	[1, 3, 5, 7, 9]
	k = 3
Yeuch
### Player 0 lost ###


Display logs

In [8]:
for i in range(1, len(log_board)):
  logging.info(f'player {log_player[i]} -> takes {log_move[i][1]} obj from row {log_move[i][0]}')
  logging.info(f'\t--> {log_board[i]}\tnim-sum = {nim_sum(log_board[i])}')

player 0 -> takes 3 obj from row 1
	--> [1, 0, 5, 7, 9]	nim-sum = 10
player 1 -> takes 1 obj from row 4
	--> [1, 0, 5, 7, 8]	nim-sum = 11
player 0 -> takes 1 obj from row 3
	--> [1, 0, 5, 6, 8]	nim-sum = 10
player 1 -> takes 1 obj from row 4
	--> [1, 0, 5, 6, 7]	nim-sum = 5
player 0 -> takes 1 obj from row 4
	--> [1, 0, 5, 6, 6]	nim-sum = 4
player 1 -> takes 1 obj from row 4
	--> [1, 0, 5, 6, 5]	nim-sum = 7
player 0 -> takes 1 obj from row 3
	--> [1, 0, 5, 5, 5]	nim-sum = 4
player 1 -> takes 1 obj from row 0
	--> [0, 0, 5, 5, 5]	nim-sum = 5
player 0 -> takes 1 obj from row 2
	--> [0, 0, 4, 5, 5]	nim-sum = 4
player 1 -> takes 1 obj from row 2
	--> [0, 0, 3, 5, 5]	nim-sum = 3
player 0 -> takes 1 obj from row 3
	--> [0, 0, 3, 4, 5]	nim-sum = 2
player 1 -> takes 1 obj from row 4
	--> [0, 0, 3, 4, 4]	nim-sum = 3
player 0 -> takes 1 obj from row 4
	--> [0, 0, 3, 4, 3]	nim-sum = 4
player 1 -> takes 3 obj from row 2
	--> [0, 0, 0, 4, 3]	nim-sum = 7
player 0 -> takes 3 obj from row 4
	--> [0, 0